In [1]:
import networkx as nx
import pickle
import os
from collections import Counter
from collections import defaultdict
import numpy as np

In [12]:
def create_dict_of_pkl(folder):
    ret = {}
    for filename in os.listdir(folder):
        if filename.endswith('.pkl'):
            filepath = os.path.join(folder, filename)
            with open(filepath, 'rb') as f:
                o = pickle.load(f)
                ret[filename] = o
    return ret

In [13]:
graphs = create_dict_of_pkl('../data/trimmed_networks_pkl')
for graph_name, graph in graphs.items():
    print(f"Graph: {graph_name}, Nodes: {graph.number_of_nodes()}, Edges: {graph.number_of_edges()}")

graph_data = create_dict_of_pkl('../data/communities_and_modularity')

Graph: 1_resource_allocation_naive.pkl, Nodes: 9213, Edges: 15588
Graph: 1_simple_disparity_filter.pkl, Nodes: 16075, Edges: 443701
Graph: 2_resource_allocation_naive.pkl, Nodes: 12770, Edges: 42691
Graph: 2_simple_disparity_filter.pkl, Nodes: 14911, Edges: 336278


In [ ]:
for name, graph in graphs.items():
    graph_nijs = [edge[2]["nij"] for edge in graph.edges(data=True)]
    print(f"average nij for {name}: {np.average(graph_nijs)}")

average nij for 1_resource_allocation_naive.pkl 10.132492970329068
average nij for 1_simple_disparity_filter.pkl 11.623410810433151
average nij for 2_resource_allocation_naive.pkl 4.940271634763646
average nij for 2_simple_disparity_filter.pkl 1.8015570450639053


### Normalized Gini Impurity
Get the proportion of matches in a community for each ground truth label for all communities and take the gini impurity divided by the maximum gini impurity it could reach (to normalize). At the end get the average of gini impurities.

In [43]:
from collections import Counter

def get_ground_truth_communities(graph, label_attribute):
    communities = defaultdict(set)

    # Iterate over nodes and group them by the label
    for node, data in graph.nodes(data=True):
        if label_attribute in data:
            label = data[label_attribute]
            communities[label].add(node)
        else:
            raise ValueError(f"Node {node} does not have the attribute '{label_attribute}'.")

    # Convert the grouped nodes into a list of sets
    return list(communities.values())

def calculate_average_gini_impurity_norm(graph, discovered_communities, ground_truth_communities):
    # Create a mapping of nodes to ground truth labels
    ground_truth_labels = {}
    for i, ground_truth_community in enumerate(ground_truth_communities):
        for node in ground_truth_community:
            ground_truth_labels[node] = i  # Assign an integer label to each ground truth community

    total_gini_impurity = 0
    total_nodes = len(graph.nodes)

    # Calculate Gini impurity for each discovered community
    for community in discovered_communities:
        # Get ground truth labels for nodes in this discovered community
        labels = [ground_truth_labels[node] for node in community if node in ground_truth_labels]
        if not labels:
            continue

        # Calculate label frequencies
        label_counts = Counter(labels)
        community_size = len(community)
        proportions = [count / community_size for count in label_counts.values()]

        # Gini impurity for this community
        gini_impurity = 1 - sum(p**2 for p in proportions)

        # Normalize the Gini impurity by its maximum value
        k = len(label_counts)  # Number of unique labels
        max_gini_impurity = 1 - 1 / k if k > 1 else 1  # Avoid division by zero for single-label communities
        normalized_gini_impurity = gini_impurity / max_gini_impurity if max_gini_impurity > 0 else 0

        # Weight by the size of the community
        total_gini_impurity += len(community) * normalized_gini_impurity

    # Average Gini impurity
    average_gini_impurity = total_gini_impurity / total_nodes
    return average_gini_impurity


In [58]:
# loop through labels and algos
community_labels = ["most_frequent_locality","parasite_group","most_frequent_animal_group"]
discovery_algorithm_labels = ["greedy_modularity","label_propagation","Louvain","Infomap"]
print('Label,Algorithm,Impurity')
for name,graph in graphs.items():
    if name == '2_simple_disparity_filter.pkl':
        largest_cc = max(nx.connected_components(graph), key=len)
        largest_component = graph.subgraph(largest_cc).copy()
        for label in community_labels:
            ground_truth_communities = get_ground_truth_communities(graph,label)
            for algorithm in discovery_algorithm_labels:
                algorithm_communities = graph_data[name][algorithm]['communities']
                print(f"{label},{algorithm},{round(calculate_average_gini_impurity_norm(largest_component,algorithm_communities,ground_truth_communities),2)}")

Label,Algorithm,Impurity
most_frequent_locality,greedy_modularity,0.8
most_frequent_locality,label_propagation,0.56
most_frequent_locality,Louvain,0.74
most_frequent_locality,Infomap,0.52
parasite_group,greedy_modularity,0.81
parasite_group,label_propagation,0.72
parasite_group,Louvain,0.76
parasite_group,Infomap,0.69
most_frequent_animal_group,greedy_modularity,0.6
most_frequent_animal_group,label_propagation,0.33
most_frequent_animal_group,Louvain,0.4
most_frequent_animal_group,Infomap,0.32


### Cut size

Function returns the generalized normalized cut size for multiple partitions of nodes.

The *generalized normalized cut size* is calculated as the sum of the cut sizes between
all pairs of partitions, normalized by the reciprocal of their volumes.

In [19]:
import itertools

def generalized_normalized_cut_size(G, partitions, weight=None):
    # Ensure the partitions cover all nodes in the graph
    all_nodes = set().union(*partitions)
    if all_nodes != set(G.nodes):
        raise ValueError("Partitions must include all nodes in the graph.")

    total_cut = 0
    combinations_count = sum(1 for _ in itertools.combinations(partitions, 2))
    # Iterate over all pairs of partitions
    for S, T in itertools.combinations(partitions, 2):
        num_cut_edges = nx.cut_size(G, S, T, weight=weight)
        volume_S = nx.volume(G, S, weight=weight)
        volume_T = nx.volume(G, T, weight=weight)
        
        # Avoid division by zero if any partition has zero volume
        if volume_S == 0 or volume_T == 0:
            raise ValueError("One of the partitions has zero volume, which is invalid.")
        
        total_cut += num_cut_edges * ((1 / volume_S) + (1 / volume_T))

    return total_cut/combinations_count

In [15]:
name = '2_simple_disparity_filter.pkl'
graph = graphs[name]
largest_cc = max(nx.connected_components(graph), key=len)
largest_component = graph.subgraph(largest_cc).copy()
discovery_algorithm_labels = ["greedy_modularity","label_propagation","Louvain","Infomap"]

In [20]:
for algorithm in discovery_algorithm_labels:
    algorithm_communities = graph_data[name][algorithm]['communities']
    cut_size = generalized_normalized_cut_size(largest_component,algorithm_communities)
    print(f'{algorithm} cut size: {cut_size}')

KeyboardInterrupt: 